# Interface de test pour la reconnaissance visuelle de la parole (VSR)

Cette application fournit une interface graphique permettant de tester un modèle de reconnaissance visuelle de la parole (VSR) qui reconnaît les mots "oui", "non", "un" et "deux" uniquement à partir des mouvements des lèvres.

## Fonctionnalités

- Affichage de la webcam en temps réel avec détection des lèvres
- Capture d'une séquence d'images des lèvres pendant la prononciation d'un mot
- Analyse de la séquence par un modèle CNN+LSTM préentraîné
- Visualisation des images capturées et des transformations CNN
- Affichage des probabilités de prédiction pour chaque mot

## Prérequis

- Python 3.x avec tkinter
- OpenCV, dlib et imutils pour la détection faciale
- TensorFlow pour le chargement et l'utilisation du modèle
- Matplotlib et seaborn pour les visualisations
- Le modèle préentraîné (`vsr_model_final.h5`) 
- Le détecteur de points faciaux (`shape_predictor_68_face_landmarks.dat`)

## Utilisation

1. Sélectionnez le mot que vous allez former avec vos lèvres
2. Cliquez sur "Enregistrer" et attendez le décompte
3. Formez le mot avec vos lèvres (sans parler) pendant l'enregistrement
4. Examinez les résultats et évaluez la prédiction
5. Consultez les statistiques pour analyser les performances globales



In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import time
import os
import pickle
import threading
import dlib
from imutils import face_utils
from PIL import Image, ImageTk

In [2]:
# Paramètres globaux
img_size = 64  # Taille des images pour le modèle VSR
max_sequence_length = 30  # Nombre max de frames pour VSR
target_words = ["oui", "non", "un", "deux"]  # Mots cibles
models_dir = "../models"  # Dossier contenant les modèles
duration = 1  # Durée d'enregistrement en secondes

In [3]:
class VSRApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Test interactif de reconnaissance visuelle de la parole (VSR)")
        self.root.geometry("1200x800")
        
        # Variables
        self.is_recording = False
        self.lip_frames = []
        self.current_frame = None
        self.video_capture = None
        self.selected_word = tk.StringVar(value=target_words[0])
        
        # Chargement du modèle VSR
        self.load_model()
        
        # Chargement du détecteur de visage
        self.setup_face_detector()
        
        # Création de l'interface
        self.create_widgets()
        
        # Démarrer la capture vidéo
        self.start_video_capture()
        
        # Résultats des tests
        self.test_results = []
        
    def load_model(self):
        """Charge le modèle VSR"""
        try:
            # Charger le modèle VSR
            vsr_model_path = os.path.join(models_dir, 'vsr_model_final.h5')
            if os.path.exists(vsr_model_path):
                from tensorflow.keras.models import load_model
                self.vsr_model = load_model(vsr_model_path)
                print("Modèle VSR chargé avec succès")
            else:
                print(f"Modèle VSR non trouvé: {vsr_model_path}")
                self.vsr_model = None
                
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur lors du chargement du modèle VSR: {e}")
            print(f"Erreur lors du chargement du modèle VSR: {e}")
            self.vsr_model = None
    
    def setup_face_detector(self):
        """Configure le détecteur de visage pour VSR"""
        try:
            self.face_detector = dlib.get_frontal_face_detector()
            predictor_path = os.path.join(models_dir, "shape_predictor_68_face_landmarks.dat")
            if os.path.exists(predictor_path):
                self.face_predictor = dlib.shape_predictor(predictor_path)
                print("Détecteur de visage chargé avec succès")
            else:
                print(f"Détecteur de points faciaux non trouvé: {predictor_path}")
                self.face_predictor = None
        except Exception as e:
            print(f"Erreur lors du chargement du détecteur de visage: {e}")
            self.face_detector = None
            self.face_predictor = None
    
    def create_widgets(self):
        """Crée l'interface utilisateur"""
        # Frame principale divisée en deux parties
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Partie gauche: Webcam et contrôles
        left_frame = ttk.Frame(main_frame)
        left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # Cadre pour la webcam
        self.webcam_frame = ttk.LabelFrame(left_frame, text="Webcam (détection des lèvres)")
        self.webcam_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        self.webcam_label = ttk.Label(self.webcam_frame)
        self.webcam_label.pack(fill=tk.BOTH, expand=True)
        
        # Contrôles
        controls_frame = ttk.LabelFrame(left_frame, text="Contrôles")
        controls_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # Sélection du mot
        word_frame = ttk.Frame(controls_frame)
        word_frame.pack(fill=tk.X, padx=5, pady=5)
        
        ttk.Label(word_frame, text="Mot à prononcer (sans parler):").pack(side=tk.LEFT, padx=5)
        word_combobox = ttk.Combobox(word_frame, textvariable=self.selected_word, values=target_words)
        word_combobox.pack(side=tk.LEFT, padx=5, fill=tk.X, expand=True)
        
        # Bouton d'enregistrement
        button_frame = ttk.Frame(controls_frame)
        button_frame.pack(fill=tk.X, padx=5, pady=5)
        
        self.record_button = ttk.Button(button_frame, text="Enregistrer", command=self.record)
        self.record_button.pack(side=tk.LEFT, padx=5, fill=tk.X, expand=True)
        
        ttk.Button(button_frame, text="Voir résultats", command=self.show_results).pack(
            side=tk.LEFT, padx=5, fill=tk.X, expand=True)
        
        # Partie droite: Résultats et visualisations
        right_frame = ttk.Frame(main_frame)
        right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        # Résultats de la prédiction
        self.result_frame = ttk.LabelFrame(right_frame, text="Résultats")
        self.result_frame.pack(fill=tk.X, padx=5, pady=5)
        
        self.result_label = ttk.Label(self.result_frame, text="Formez un mot avec vos lèvres (sans parler)...", font=("Arial", 14))
        self.result_label.pack(padx=10, pady=10)
        
        self.confidence_label = ttk.Label(self.result_frame, text="")
        self.confidence_label.pack(padx=10, pady=5)
        
        # Évaluation
        eval_frame = ttk.Frame(self.result_frame)
        eval_frame.pack(fill=tk.X, padx=5, pady=5)
        
        ttk.Button(eval_frame, text="Correct", command=lambda: self.evaluate(True)).pack(
            side=tk.LEFT, padx=5, fill=tk.X, expand=True)
        
        ttk.Button(eval_frame, text="Incorrect", command=lambda: self.evaluate(False)).pack(
            side=tk.LEFT, padx=5, fill=tk.X, expand=True)
        
        # Frame pour les visualisations
        self.viz_frame = ttk.LabelFrame(right_frame, text="Visualisations")
        self.viz_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Tabs pour différentes visualisations
        self.viz_tabs = ttk.Notebook(self.viz_frame)
        self.viz_tabs.pack(fill=tk.BOTH, expand=True)
        
        # Onglet séquence d'images
        self.sequence_tab = ttk.Frame(self.viz_tabs)
        self.viz_tabs.add(self.sequence_tab, text="Séquence d'images")
        
        # Onglet transformations
        self.transform_tab = ttk.Frame(self.viz_tabs)
        self.viz_tabs.add(self.transform_tab, text="Transformations CNN")
        
        # Onglet probabilités
        self.proba_tab = ttk.Frame(self.viz_tabs)
        self.viz_tabs.add(self.proba_tab, text="Probabilités")
        
        # Barre de statut
        self.status_bar = ttk.Label(self.root, text="Prêt", relief=tk.SUNKEN, anchor=tk.W)
        self.status_bar.pack(side=tk.BOTTOM, fill=tk.X)
    
    def start_video_capture(self):
        """Démarre la capture vidéo depuis la webcam"""
        self.video_capture = cv2.VideoCapture(0)
        if not self.video_capture.isOpened():
            messagebox.showerror("Erreur", "Impossible d'accéder à la webcam")
            return
            
        self.update_webcam()
    
    def update_webcam(self):
        """Met à jour l'affichage de la webcam"""
        ret, frame = self.video_capture.read()
        if ret:
            self.current_frame = frame.copy()
            
            # Si enregistrement vidéo en cours, traiter le frame pour VSR
            if self.is_recording:
                lip_region = self.extract_lip_region(frame)
                if lip_region is not None:
                    processed_lip = self.preprocess_lip_region(lip_region)
                    if processed_lip is not None:
                        self.lip_frames.append(processed_lip.squeeze())
            
            # Afficher les points du visage si disponible
            if self.face_detector is not None and self.face_predictor is not None:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = self.face_detector(gray, 0)
                for face in faces:
                    shape = self.face_predictor(gray, face)
                    shape = face_utils.shape_to_np(shape)
                    
                    # Dessiner les points des lèvres (48-68)
                    for i in range(48, 68):
                        cv2.circle(frame, (shape[i][0], shape[i][1]), 2, (0, 255, 0), -1)
                    
                    # Dessiner un rectangle autour des lèvres
                    lips_points = shape[48:68]
                    x, y = lips_points.min(axis=0)
                    w, h = lips_points.max(axis=0) - lips_points.min(axis=0)
                    margin = 10
                    cv2.rectangle(frame, (x-margin, y-margin), (x+w+margin, y+h+margin), (0, 0, 255), 2)
            
            # Convertir pour affichage dans Tkinter
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame_rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            self.webcam_label.imgtk = imgtk
            self.webcam_label.configure(image=imgtk)
        
        # Rappel pour mise à jour continue
        self.root.after(10, self.update_webcam)
    
    def extract_lip_region(self, frame):
        """Extrait la région des lèvres d'une image"""
        if self.face_detector is None or self.face_predictor is None:
            return None
            
        # Convertir en niveaux de gris pour la détection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Détecter les visages
        faces = self.face_detector(gray, 0)
        
        if len(faces) == 0:
            return None
        
        # Prendre le premier visage détecté
        face = faces[0]
        
        # Prédire les points de repère
        shape = self.face_predictor(gray, face)
        shape = face_utils.shape_to_np(shape)
        
        # Les points des lèvres sont les points 48-68 dans le modèle à 68 points
        lips_points = shape[48:68]
        
        # Calculer le rectangle englobant pour les lèvres
        x, y = lips_points.min(axis=0)
        w, h = lips_points.max(axis=0) - lips_points.min(axis=0)
        
        # Ajouter une marge
        margin = 10
        x = max(0, x - margin)
        y = max(0, y - margin)
        w = min(frame.shape[1] - x, w + 2 * margin)
        h = min(frame.shape[0] - y, h + 2 * margin)
        
        # Extraire la région des lèvres
        lip_region = frame[y:y+h, x:x+w]
        
        return lip_region
    
    def preprocess_lip_region(self, lip_region):
        """Prétraite une région de lèvres pour la prédiction"""
        if lip_region is None:
            return None
        
        # Convertir en niveaux de gris
        gray = cv2.cvtColor(lip_region, cv2.COLOR_BGR2GRAY)
        
        # Redimensionner
        resized = cv2.resize(gray, (img_size, img_size))
        
        # Normaliser
        normalized = resized / 255.0
        
        # Ajouter les dimensions pour le modèle
        processed = normalized.reshape(1, img_size, img_size, 1)
        
        return processed
    
    def record(self):
        """Enregistre la vidéo pour la prédiction"""
        # Vérifier que le modèle est chargé
        if self.vsr_model is None or self.face_predictor is None:
            messagebox.showerror("Erreur", "Le modèle VSR ou le détecteur de visage n'est pas chargé")
            return
        
        # Désactiver le bouton pendant l'enregistrement
        self.record_button.config(state=tk.DISABLED)
        self.status_bar.config(text="Préparation de l'enregistrement...")
        
        # Lancer l'enregistrement dans un thread séparé
        threading.Thread(target=self._record_thread).start()
    
    def _record_thread(self):
        """Fonction d'enregistrement exécutée dans un thread séparé"""
        # Countdown avant enregistrement
        for i in range(3, 0, -1):
            self.status_bar.config(text=f"Préparez-vous... {i}")
            time.sleep(1)
        
        self.status_bar.config(text="Formez le mot avec vos lèvres maintenant! (sans parler)")
        
        # Démarrer l'enregistrement vidéo
        self.is_recording = True
        self.lip_frames = []
        
        # Attendre la durée d'enregistrement
        time.sleep(duration)
        
        # Arrêter l'enregistrement vidéo
        self.is_recording = False
        
        # Traiter les résultats
        self.process_recording()
        
        # Réactiver le bouton
        self.record_button.config(state=tk.NORMAL)
    
    def process_recording(self):
        """Traite l'enregistrement et affiche les résultats"""
        self.status_bar.config(text="Traitement de l'enregistrement...")
        
        # Vérifier s'il y a assez de frames
        if len(self.lip_frames) < 5:
            self.result_label.config(text="Pas assez d'images des lèvres détectées")
            self.confidence_label.config(text="Veuillez réessayer")
            self.status_bar.config(text="Prêt")
            return
        
        # Prédiction VSR
        predicted_word, confidence, probabilities = self.predict_vsr()
        
        # Afficher les résultats
        result_text = f"Mot prédit: {predicted_word}"
        self.result_label.config(text=result_text)
        self.confidence_label.config(text=f"Confiance: {confidence:.2f}")
        
        # Stocker le résultat temporaire
        self.current_result = {
            "actual": self.selected_word.get(),
            "predicted": predicted_word,
            "confidence": confidence,
            "probabilities": probabilities.tolist()  # Pour la sauvegarde JSON
        }
        
        # Afficher les visualisations
        self.visualize_sequence()
        self.visualize_probabilities(probabilities)
        
        self.status_bar.config(text="Prêt")
    
    def predict_vsr(self):
        """Prédit le mot à partir de la séquence d'images des lèvres"""
        try:
            # Préparer la séquence pour le modèle
            lip_sequence = self.prepare_lip_sequence()
            
            # Prédiction
            prediction = self.vsr_model.predict(lip_sequence)
            predicted_class = np.argmax(prediction[0])
            confidence = prediction[0][predicted_class]
            
            # Résultat
            predicted_word = target_words[predicted_class]
            
            return predicted_word, confidence, prediction[0]
            
        except Exception as e:
            print(f"Erreur lors de la prédiction VSR: {e}")
            return "Erreur", 0.0, np.zeros(len(target_words))
    
    def prepare_lip_sequence(self):
        """Prépare la séquence d'images des lèvres pour la prédiction"""
        # Padding si nécessaire
        sequence = self.lip_frames.copy()
        if len(sequence) < max_sequence_length:
            padding = [np.zeros((img_size, img_size)) for _ in range(max_sequence_length - len(sequence))]
            sequence.extend(padding)
        
        # Limiter à max_sequence_length frames
        sequence = sequence[:max_sequence_length]
        
        # Convertir en tableau numpy
        return np.array(sequence).reshape(1, max_sequence_length, img_size, img_size, 1)
    
    def visualize_sequence(self):
        """Affiche la séquence d'images des lèvres"""
        # Nettoyer les onglets
        for widget in self.sequence_tab.winfo_children():
            widget.destroy()
        for widget in self.transform_tab.winfo_children():
            widget.destroy()
        
        # Séquence d'images
        fig_seq = plt.Figure(figsize=(10, 3))
        
        # Sélectionner quelques frames à afficher
        num_display = min(6, len(self.lip_frames))
        indices = np.linspace(0, len(self.lip_frames)-1, num_display, dtype=int)
        
        for i, idx in enumerate(indices):
            ax = fig_seq.add_subplot(1, num_display, i+1)
            ax.imshow(self.lip_frames[idx], cmap='gray')
            ax.set_title(f"Frame {idx+1}")
            ax.axis('off')
        
        fig_seq.tight_layout()
        fig_seq.suptitle('Séquence d\'images des lèvres', y=0.98)
        
        canvas_seq = FigureCanvasTkAgg(fig_seq, master=self.sequence_tab)
        canvas_seq.draw()
        canvas_seq.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        
        # Visualisation des transformations (si modèle VSR disponible)
        if self.vsr_model is not None and len(self.lip_frames) >= 5:
            # Si possible, visualiser les premières couches convolutives
            try:
                from tensorflow.keras.models import Model
                
                # Créer un modèle pour extraire les caractéristiques
                feature_model = Model(
                    inputs=self.vsr_model.input,
                    outputs=self.vsr_model.get_layer(index=1).output  # Première couche conv
                )
                
                # Préparer la séquence
                lip_sequence = self.prepare_lip_sequence()
                
                # Prédire pour obtenir les cartes de caractéristiques
                features = feature_model.predict(lip_sequence)
                
                # Sélectionner la première frame
                middle_frame_idx = 0
                feature_maps = features[0][middle_frame_idx]
                
                # Afficher
                fig_trans = plt.Figure(figsize=(10, 5))
                
                # Image originale
                ax_orig = fig_trans.add_subplot(2, 4, 1)
                ax_orig.imshow(self.lip_frames[0], cmap='gray')
                ax_orig.set_title('Image originale')
                ax_orig.axis('off')
                
                # Cartes de caractéristiques
                num_features = min(7, feature_maps.shape[2])
                for i in range(num_features):
                    ax = fig_trans.add_subplot(2, 4, i+2)
                    ax.imshow(feature_maps[:, :, i], cmap='viridis')
                    ax.set_title(f'Filtre {i+1}')
                    ax.axis('off')
                
                fig_trans.tight_layout()
                fig_trans.suptitle('Transformations appliquées par la première couche CNN', y=0.98)
                
                canvas_trans = FigureCanvasTkAgg(fig_trans, master=self.transform_tab)
                canvas_trans.draw()
                canvas_trans.get_tk_widget().pack(fill=tk.BOTH, expand=True)
                
            except Exception as e:
                print(f"Erreur lors de la visualisation des transformations: {e}")
                error_label = ttk.Label(self.transform_tab, text=f"Erreur: {e}")
                error_label.pack(padx=10, pady=10)
    
    def visualize_probabilities(self, probabilities):
        """Affiche les probabilités pour chaque classe"""
        # Nettoyer l'onglet
        for widget in self.proba_tab.winfo_children():
            widget.destroy()
        
        # Créer le graphique
        fig = plt.Figure(figsize=(8, 4))
        ax = fig.add_subplot(111)
        
        bars = ax.bar(target_words, probabilities, color='skyblue')
        
        # Mettre en évidence la classe prédite
        predicted_class = np.argmax(probabilities)
        bars[predicted_class].set_color('navy')
        
        # Ajouter les valeurs sur les barres
        for i, p in enumerate(probabilities):
            ax.text(i, p + 0.01, f'{p:.3f}', ha='center')
        
        ax.set_ylim(0, 1.1)
        ax.set_ylabel('Probabilité')
        ax.set_title('Probabilités pour chaque mot')
        
        # Ajouter au canvas
        canvas = FigureCanvasTkAgg(fig, master=self.proba_tab)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    def evaluate(self, is_correct):
        """Enregistre l'évaluation de l'utilisateur"""
        if hasattr(self, 'current_result'):
            # Ajouter le résultat aux résultats des tests
            self.current_result["correct"] = is_correct
            self.test_results.append(self.current_result)
            
            # Message de confirmation
            self.status_bar.config(text=f"Résultat enregistré. Total: {len(self.test_results)} tests.")
            
            # Effacer le résultat actuel
            delattr(self, 'current_result')
        else:
            messagebox.showinfo("Info", "Aucun résultat à évaluer. Veuillez d'abord enregistrer un mot.")
    
    def show_results(self):
        """Affiche les résultats des tests"""
        if not self.test_results:
            messagebox.showinfo("Info", "Aucun résultat enregistré")
            return
        
        # Créer une nouvelle fenêtre pour les résultats
        results_window = tk.Toplevel(self.root)
        results_window.title("Résultats des tests VSR")
        results_window.geometry("800x600")
        
        # Frame principale
        main_frame = ttk.Frame(results_window)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Tableau des résultats
        table_frame = ttk.LabelFrame(main_frame, text="Résultats détaillés")
        table_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Treeview pour afficher les résultats
        columns = ("actual", "predicted", "confidence", "correct")
        tree = ttk.Treeview(table_frame, columns=columns, show="headings")
        
        # Définir les en-têtes
        tree.heading("actual", text="Mot formé")
        tree.heading("predicted", text="Mot prédit")
        tree.heading("confidence", text="Confiance")
        tree.heading("correct", text="Correct")
        
        # Définir les largeurs de colonnes
        tree.column("actual", width=150)
        tree.column("predicted", width=150)
        tree.column("confidence", width=100)
        tree.column("correct", width=100)
        
        # Ajouter les données
        for i, result in enumerate(self.test_results):
            tree.insert("", tk.END, values=(
                result["actual"],
                result["predicted"],
                f"{result['confidence']:.2f}",
                "Oui" if result["correct"] else "Non"
            ))
        
        # Ajouter scrollbar
        scrollbar = ttk.Scrollbar(table_frame, orient=tk.VERTICAL, command=tree.yview)
        tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        tree.pack(fill=tk.BOTH, expand=True)
        
        # Statistiques
        stats_frame = ttk.LabelFrame(main_frame, text="Statistiques")
        stats_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # Calculer l'exactitude
        correct_count = sum(1 for r in self.test_results if r["correct"])
        accuracy = correct_count / len(self.test_results) if self.test_results else 0
        
        # Calculer les statistiques par mot
        word_stats = {}
        for word in target_words:
            word_results = [r for r in self.test_results if r["actual"] == word]
            if word_results:
                word_correct = sum(1 for r in word_results if r["correct"])
                word_accuracy = word_correct / len(word_results)
                word_stats[word] = {
                    "count": len(word_results),
                    "correct": word_correct,
                    "accuracy": word_accuracy
                }
        
        # Afficher les statistiques
        ttk.Label(stats_frame, text=f"Exactitude globale: {accuracy:.2%} ({correct_count}/{len(self.test_results)})").pack(anchor=tk.W, padx=5, pady=2)
        
        for word, stats in word_stats.items():
            ttk.Label(stats_frame, text=f"Mot '{word}': {stats['accuracy']:.2%} ({stats['correct']}/{stats['count']})").pack(anchor=tk.W, padx=5, pady=2)
        
        # Matrice de confusion
        viz_frame = ttk.LabelFrame(main_frame, text="Matrice de confusion")
        viz_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        try:
            from sklearn.metrics import confusion_matrix
            import seaborn as sns
            
            # Collecter les étiquettes réelles et prédites
            y_true = []
            y_pred = []
            
            for result in self.test_results:
                if result["actual"] in target_words and result["predicted"] in target_words:
                    y_true.append(target_words.index(result["actual"]))
                    y_pred.append(target_words.index(result["predicted"]))
            
            if len(y_true) > 0:
                # Calculer la matrice de confusion
                cm = confusion_matrix(y_true, y_pred, labels=range(len(target_words)))
                
                # Afficher la matrice
                fig = plt.Figure(figsize=(6, 4))
                ax = fig.add_subplot(111)
                sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_words, yticklabels=target_words, ax=ax)
                ax.set_xlabel('Prédit')
                ax.set_ylabel('Réel')
                ax.set_title('Matrice de confusion')
                
                canvas = FigureCanvasTkAgg(fig, master=viz_frame)
                canvas.draw()
                canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
            else:
                ttk.Label(viz_frame, text="Pas assez de données pour afficher la matrice de confusion").pack(padx=10, pady=10)
                
        except Exception as e:
            ttk.Label(viz_frame, text=f"Erreur lors de la création de la matrice de confusion: {e}").pack(padx=10, pady=10)
        
        # Bouton pour sauvegarder les résultats
        ttk.Button(main_frame, text="Sauvegarder les résultats", 
                  command=lambda: self.save_results(results_window)).pack(pady=10)
    
    def save_results(self, parent_window):
        """Sauvegarde les résultats des tests dans un fichier"""
        try:
            import json
            from datetime import datetime
            
            # Générer un nom de fichier avec la date et l'heure
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"vsr_test_results_{timestamp}.json"
            
            # Sauvegarder en JSON
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.test_results, f, ensure_ascii=False, indent=4)
            
            messagebox.showinfo("Succès", f"Résultats sauvegardés dans {filename}", parent=parent_window)
            
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur lors de la sauvegarde: {e}", parent=parent_window)
    
    def on_closing(self):
        """Gestion de la fermeture de l'application"""
        if self.video_capture is not None:
            self.video_capture.release()
        self.root.destroy()

In [4]:
# Point d'entrée principal
if __name__ == "__main__":
    root = tk.Tk()
    app = VSRApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()

Modèle VSR chargé avec succès
Détecteur de visage chargé avec succès
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
